In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/combine_data.csv")

In [3]:
df.head()

,Unnamed: 0,Country,year,Happiness Score,Log GDP per capita,Social support,Health,Freedom to make life choices,Generosity,Corruption,...,Negative affect,Confidence in national government,continent,Country Code,Time Code,CO2 emissions,Population,Unemployment,Children out of school,Adjusted net national income
0,0,Afghanistan,2008,3.723590,7.302574,0.450662,50.500000,0.718114,0.173169,0.881686,...,0.258195,0.612072,Asia,AFG,YR2008,3559.999943,26427199.0,2.49,NaN,NaN
1,1,Afghanistan,2009,4.401778,7.472446,0.552308,50.799999,0.678896,0.195469,0.850035,...,0.237092,0.611545,Asia,AFG,YR2009,4880.000114,27385307.0,NaN,NaN,1.129747e+10
2,2,Afghanistan,2010,4.758381,7.579183,0.539075,51.099998,0.600127,0.125859,0.706766,...,0.275324,0.299357,Asia,AFG,YR2010,7110.000134,28189672.0,NaN,NaN,1.448363e+10
3,3,Afghanistan,2011,3.831719,7.552006,0.521104,51.400002,0.495901,0.167723,0.731109,...,0.267175,0.307386,Asia,AFG,YR2011,8930.000305,29249157.0,NaN,NaN,1.626732e+10
4,4,Afghanistan,2012,3.782938,7.637953,0.520637,51.700001,0.530935,0.241247,0.775620,...,0.267919,0.435440,Asia,AFG,YR2012,8079.999924,30466479.0,1.69,NaN,1.822551e+10


In [4]:
# Some data do not have continent value, drop these value
df3 = pd.DataFrame({'Affect': df['Positive affect'].tolist() + df['Negative affect'].tolist(),
        'Situation': ['Positive']*len(df['Positive affect'].tolist()) + ['Negative']*len(df['Negative affect'].tolist()),
        'Country': df['Country'].tolist()*2,
        'year': df['year'].tolist()*2,
        'continent': df['continent'].tolist()*2})
df3 = df3.dropna(subset=['continent'])

In [5]:
# import the plotly library for data visualization
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

pio.renderers.default = "plotly_mimetype+notebook_connected"

In [6]:
# INITIALIZE GRAPH OBJECT
import numpy as np
import plotly.io as pio
fig = go.Figure()


# Set the continent values
cont = ['Oceania',
        'North America',
        'Europe',
        'South America',
        'Asia',
        'Africa']

# Set the color scheme
scheme = ['rgb(166, 216, 84)',
          'rgb(231, 138, 195)',
          'rgb(141, 160, 203)',
          'rgb(255, 217, 47)',
          'rgb(252, 141, 98)',
          'rgb(102, 194, 165)']
# add trace for every situation
j = 0
for i in cont:
    df4 = df3.query("continent==@i")
    df4_pos = df4[df4["Situation"]=='Positive']
    # get the mean value of the affect based on the year and continent
    df4_pos = df4_pos.groupby('year')['Affect'].mean()
    df4_pos = df4_pos.reset_index(drop=False)
    fig.add_trace(  # Add A trace to the figure
    go.Scatter(  # Specify the type of the trace
        x=df4_pos["year"],  # Data-x
        y=df4_pos["Affect"],  # Data-y
        line =dict(color = scheme[j]),
        name = i,
        opacity=1.0,
        visible=True,  # Specify whether or not to make data-visible when rendered
        legendgroup=cont[j],
        hovertemplate = 'Continent: '+i+
                        '<br>Year: %{x}<br>Positive Value: %{y}<extra></extra>',
    )
)
    
    df4_neg = df4[df4["Situation"]=='Negative']
    # get the mean value of the affect based on the year and continent
    df4_neg = df4_neg.groupby('year')['Affect'].mean()
    df4_neg = df4_neg.reset_index(drop=False)
    fig.add_trace(  # Add A trace to the figure
    go.Scatter(  # Specify the type of the trace
        x=df4_neg["year"],  # Data-x
        y=df4_neg["Affect"],  # Data-y
        line= dict(color = scheme[j],
                   dash = 'dash'),
        name=i+'_Negative',
        opacity=1.0,
        visible=True,  # Specify whether or not to make data-visible when rendered
        legendgroup=i,
        showlegend=False,
        hovertemplate = 'Continent: '+i+
                        '<br>Year: %{x}<br>Negative Value: %{y}<extra></extra>',
    )

)
    j+=1

# update the layout
fig.update_layout(
    xaxis = dict(
        tickvals = list(set(df3['year']))
    ),
    plot_bgcolor = 'rgba(0,0,0,0)',
    title = 'Positive Affect & Negative Affect from 2008 to 2021 among different continents',
    xaxis_title = 'Time (Year)',
    yaxis_title = 'Positive Affect & Negative Affect Value',
    yaxis_range=[0.05,0.85],
    boxmode = 'group',
    legend_title_text = "Continent"
)
# change the y-axis
fig.update_yaxes(
                 showline=True,
                 linewidth=1,
                 showgrid = True, 
                 zeroline=True,
                 gridcolor='lightgrey',
                 layer='below traces',
                 tickvals=[0.1, 0.2, 0.3, 0.4,
                           0.5, 0.6, 0.7, 0.8])
fig.show()
fig.write_html("../img/lineplot_affect.html")
